<a href="https://colab.research.google.com/github/punkmic/Web-Scraping-with-persistence-/blob/master/Web_Scraping_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This project implements a simple Web Scraping example and persists the data in different ways (CSV, JSON, and SQL database) for future work**

## **Set up the enviroment and install requirements**

In [ ]:
!python3 -m venv venv
!source venv/bin/activate
!pip install -r requirements.txt

## **Parse the local files** 

In [5]:
import scrapy 
import os

current_dir = os.path.abspath('')
url = os.path.join(current_dir,"/content/html/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")

with open(url) as _f:
  url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding="utf-8")
print(response.url)

table = response.xpath('//table')[1].xpath('tbody')

## **Extract data and persist it in a json file**

In [9]:
import json

scrapped_data = {}

# extract data
for tr in table.xpath('tr'):
  medal = tr.xpath('td/b/text()').extract()[0]
  athlete = tr.xpath('td/a/text()').extract()[0]
  scrapped_data[medal] = athlete

print(scrapped_data)

# save to json file
with open('1992_results.json', 'w') as _f:
  json.dump(scrapped_data, _f)


{'Gold': 'Steve Smith', 'Silver': 'Tim Forsyth', 'Bronze': 'Takahiro Kimino'}


## **Extract data and persist it in a csv file**

In [10]:
import csv

column_names = ['Medal', 'Athlete']
rows = []

for tr in table.xpath('tr'):
  medal = tr.xpath('td/b/text()').extract()[0]
  athlete = tr.xpath('td/a/text()').extract()[0]
  rows.append([medal, athlete])

with open('1992_results.csv', 'w') as _f:
  writer = csv.writer(_f)

  # write the column names 
  writer.writerow(column_names)

  # write the rows
  writer.writerows(rows)



## **Extract data and persist it in a SQL database**

In [11]:
import sqlite3

connection = sqlite3.connect('1992_results.db')
db_table = 'CREATE TABLE results(id integer primary key, medal TEXT, athlete TEXT)'
cursor = connection.cursor()
cursor.execute(db_table)
connection.commit()


In [14]:
connection = sqlite3.connect('1992_results.db')
cursor = connection.cursor()

query = 'INSERT INTO results(medal, athlete) VALUES(?, ?)'

for tr in table.xpath('tr'):
  medal = tr.xpath('td/b/text()').extract()[0]
  athlete = tr.xpath('td/a/text()').extract()[0]
  cursor.execute(query, (medal, athlete))
  connection.commit()